In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm

DATASET_DIR = "dataset/train"
labels = ["50","100","200","500","1000"]

X = []
y = []

for label in labels:
  folder = os.path.join(DATASET_DIR, label)
  for img_name in tqdm(os.listdir(folder), desc=f"Load {label}"):
    path = os.path.join(folder, img_name)
    img = cv2.imread(path)

    img = cv2.resize(img, (128,128))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    X.append(img.flatten())
    y.append(label)

X = np.array(X)
y = np.array(y)

print("Dataset:", X.shape, "Labels:", y.shape)


Load 1000: 100%|██████████| 28/28 [00:00<00:00, 876.32it/s]

Dataset: (140, 16384) Labels: (140,)


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.2, random_state=42
)

model = SVC(kernel="linear", probability=True)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

         100       0.33      1.00      0.50         3
        1000       1.00      0.29      0.44         7
         200       1.00      1.00      1.00         7
          50       1.00      0.67      0.80         6
         500       0.67      0.80      0.73         5

    accuracy                           0.71        28
   macro avg       0.80      0.75      0.69        28
weighted avg       0.87      0.71      0.72        28



In [3]:
import joblib

joblib.dump(model, "model_koin.pkl")
print("Model saved as model_koin.pkl")

model = joblib.load("model_koin.pkl")
print("Model loaded.")


Model saved as model_koin.pkl
Model loaded.


In [4]:
def detect_and_label(img):
  import cv2
  import numpy as np

  img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  circles = cv2.HoughCircles(
    gray,
    cv2.HOUGH_GRADIENT,
    dp=1,
    minDist=50,
    param1=100,
    param2=30,
    minRadius=20,
    maxRadius=200
  )

  output = img.copy()

  if circles is not None:
    circles = np.uint16(np.around(circles))
    for (x, y, r) in circles[0, :]:
      roi = gray[y-r:y+r, x-r:x+r]
      if roi.size == 0:
        continue

      roi_resized = cv2.resize(roi, (128,128)).flatten().reshape(1, -1)
      pred = model.predict(roi_resized)[0]

      cv2.circle(output, (x, y), r, (255, 0, 0), 3)   # merah
      cv2.putText(output, str(pred), (x - 20, y - r - 10),
                  cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

  output = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)
  return output


In [ ]:
import gradio as gr

gr.Interface(
  fn=detect_and_label,
  inputs=gr.Image(type="pil", label="Upload Foto Koin"),
  outputs=gr.Image(type="numpy", label="Hasil Deteksi"),
  title="Klasifikasi Koin Rupiah",
  description="Upload gambar, sistem akan mendeteksi koin dan memberi label nominal."
).launch()


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22152\700476529.py:24: RuntimeWarning: overflow encountered in scalar subtract
  roi = gray[y-r:y+r, x-r:x+r]
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22152\700476529.py:24: RuntimeWarning: overflow encountered in scalar subtract
  roi = gray[y-r:y+r, x-r:x+r]
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22152\700476529.py:24: RuntimeWarning: overflow encountered in scalar subtract
  roi = gray[y-r:y+r, x-r:x+r]
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22152\700476529.py:24: RuntimeWarning: overflow encountered in scalar subtract
  roi = gray[y-r:y+r, x-r:x+r]
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22152\700476529.py:24: RuntimeWarning: overflow encountered in scalar subtract
  roi = gray[y-r:y+r, x-r:x+r]
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22152\700476529.py:24: RuntimeWarning: overflow encountered in scalar subtract
  roi = gray[y-r:y+r, x-r:x+r]
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22152\700476529.py:24: Ru